In [1]:
'''
验证光大证券报告中的数据
'''
import pandas
import os
from datetime import datetime
import numpy

current_path = os.path.abspath('')
df = pandas.read_excel(current_path + '/../data/data.xlsx')
df['上市公告日期'] = pandas.to_datetime(df['上市公告日期'],format="%Y-%m-%d")
# 2019年至2022年11月18日 共558只可转债
start = pandas.Timestamp("2019-01-01")
end = pandas.Timestamp("2022-11-18")
df = df[(df['上市公告日期']>=start) & (df['上市公告日期']<=end)]
count = df['上市公告日期'].count()
print("可转债数量: {}".format(count))


可转债数量: 558


In [2]:
# 转债发行及上市首日表现情况
year = {
    '整体' : [pandas.Timestamp("2019-01-01"), pandas.Timestamp("2022-11-18")],
    '2019' : [pandas.Timestamp("2019-01-01"), pandas.Timestamp("2019-12-31")],
    '2020' : [pandas.Timestamp("2020-01-01"), pandas.Timestamp("2020-12-31")],
    '2021' : [pandas.Timestamp("2021-01-01"), pandas.Timestamp("2021-12-31")],
    '2022' : [pandas.Timestamp("2022-01-01"), pandas.Timestamp("2022-12-31")]
}
for d, v in year.items():
    tf = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]
    count = tf['上市公告日期'].count()
    total = tf['发行金额(万元)'].sum()
    mean = tf['上市首日收盘价'].apply(lambda x : (x-100)/100).mean()
    median = tf['上市首日收盘价'].apply(lambda x : (x-100)/100).median()
    print("{}\t 发行规模(亿元): {:.2f}\t 发行只数: {}\t 涨跌幅平均数: {:.2%}\t 涨跌幅中位数: {:.2%}".format(d, total/10000, count, mean, median))


整体	 发行规模(亿元): 10048.21	 发行只数: 558	 涨跌幅平均数: 19.82%	 涨跌幅中位数: 17.42%
2019	 发行规模(亿元): 2477.81	 发行只数: 106	 涨跌幅平均数: 9.44%	 涨跌幅中位数: 8.77%
2020	 发行规模(亿元): 2466.81	 发行只数: 204	 涨跌幅平均数: 19.10%	 涨跌幅中位数: 17.19%
2021	 发行规模(亿元): 2697.81	 发行只数: 121	 涨跌幅平均数: 19.27%	 涨跌幅中位数: 18.48%
2022	 发行规模(亿元): 2405.78	 发行只数: 127	 涨跌幅平均数: 30.18%	 涨跌幅中位数: 25.90%


In [3]:
# 股东配售比例
df['股东配售比例'] = df.apply(lambda x : x['向原股东配售数量（张）']/(x['发行金额(万元)']*10000/100), axis=1)
for d, v in year.items():
    tf = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]
    ratio = tf['股东配售比例'].mean()
    ratio2 = numpy.mean([ float(x.split('%')[0]) for x in tf['股东配售率'] ])
    #print("{}\t 股东平均配售比例: {:.2%}\t {:.2f}%".format(d, ratio, ratio2))
    print("{}\t 股东平均配售比例: {:.2%}".format(d, ratio))

整体	 股东平均配售比例: 60.27%
2019	 股东平均配售比例: 43.68%
2020	 股东平均配售比例: 64.17%
2021	 股东平均配售比例: 59.58%
2022	 股东平均配售比例: 68.52%


In [4]:
t = {
    'T-2' : 'T-2日',
    'T-1' : 'T-1日', 
}
# 从平均数看， 如在T-2日参与转债抢权，则一手转债成本1000元，平均可盈利93.6元，其中可转债收益201.2元，股票亏损107.5元
df['可转债上市当天成交均价'] = df.apply(lambda x : x['上市首日成交金额(万)']/(x['发行金额(万元)']*10*x['上市首日换手率(%)']) * 100 * 1000, axis=1)
for key, value in t.items():
    print(value)
    for d, v in year.items():
        tf = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]
        money_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * x[value + '均价'], axis=1).mean()
        profit_cb = tf['可转债上市当天成交均价'].apply(lambda x : x-100).mean() * 10
        profit_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x[value + '均价']), axis=1).mean()
        profit_avg = profit_cb + profit_st
        print("{}\t 每手正股参与金额: {:.2f}\t 每手可转债净收益: {:.2f}\t 转债收益: {:.2f}    \t 股票收益: {:.2f}".format(d, money_st, profit_avg, profit_cb, profit_st))


T-2日
整体	 每手正股参与金额: 14898.16	 每手可转债净收益: 93.64	 转债收益: 201.15    	 股票收益: -107.52
2019	 每手正股参与金额: 12428.11	 每手可转债净收益: 170.01	 转债收益: 93.48    	 股票收益: 76.52
2020	 每手正股参与金额: 15345.84	 每手可转债净收益: 96.93	 转债收益: 200.31    	 股票收益: -103.38
2021	 每手正股参与金额: 16870.29	 每手可转债净收益: 74.58	 转债收益: 192.88    	 股票收益: -118.30
2022	 每手正股参与金额: 14361.73	 每手可转债净收益: 42.76	 转债收益: 300.26    	 股票收益: -257.50
T-1日
整体	 每手正股参与金额: 14943.98	 每手可转债净收益: 47.82	 转债收益: 201.15    	 股票收益: -153.33
2019	 每手正股参与金额: 12477.50	 每手可转债净收益: 120.62	 转债收益: 93.48    	 股票收益: 27.13
2020	 每手正股参与金额: 15410.01	 每手可转债净收益: 32.76	 转债收益: 200.31    	 股票收益: -167.55
2021	 每手正股参与金额: 16881.11	 每手可转债净收益: 63.75	 转债收益: 192.88    	 股票收益: -129.13
2022	 每手正股参与金额: 14408.42	 每手可转债净收益: -3.94	 转债收益: 300.26    	 股票收益: -304.20


In [13]:
# 股东配售比例对抢权配售影响
old_value = {
    '小于40%' : [0,0.4],
    '40%-50%' : [0.4, 0.5],
    '50%-60%' : [0.5, 0.6],
    '60%-70%' : [0.6, 0.7],
    '70%-80%' : [0.7, 0.8],
    '80%-90%' : [0.8, 0.9],
    '90%以上' : [0.9, 1],
}

print("{:10s}\t {:<10s}\t {:>10s}\t {:10s}\t {:10s}\t {:10s}\t {:10s}".format("股东配售比例","T-2日至T日", "T日","转债盈亏","正股盈亏","总收益", "数量"))
for d, v in old_value.items():
    tf = df[(df['股东配售比例'] < v[1]) & (df['股东配售比例'] >= v[0])]
    limit_avg_1 = tf.apply(lambda x : (x['T日均价']-x['T-2日均价'])/x['T-2日均价'], axis=1).mean()
    limit_avg_2 = tf['T日涨跌幅'].mean()
    count = tf['T日涨跌幅'].count()
    profit_cb = tf['可转债上市当天成交均价'].apply(lambda x : x-100).mean() * 10
    profit_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x['T-2日均价']), axis=1).mean()
    profit_avg = profit_cb + profit_st
    print("{:10s}\t {:<10.2%}\t {:>10.2f}%\t {:<10.2f}\t {:<10.2f}\t {:<10.2f}\t {}".format(d, limit_avg_1, limit_avg_2, profit_cb, profit_st, profit_avg, count))

股东配售比例    	 T-2日至T日   	         T日	 转债盈亏      	 正股盈亏      	 总收益       	 数量        
小于40%     	 -0.84%    	      -1.06%	 120.00    	 -126.33   	 -6.33     	 118
40%-50%   	 -0.67%    	      -0.87%	 151.20    	 -7.11     	 144.09    	 44
50%-60%   	 -1.52%    	      -1.86%	 173.90    	 -102.63   	 71.27     	 63
60%-70%   	 -1.87%    	      -2.13%	 246.16    	 -237.10   	 9.05      	 105
70%-80%   	 -0.99%    	      -1.65%	 233.63    	 -78.71    	 154.92    	 119
80%-90%   	 -1.18%    	      -1.80%	 240.54    	 -35.43    	 205.12    	 92
90%以上     	 -1.52%    	      -1.61%	 276.27    	 -46.33    	 229.95    	 17


In [6]:
# T-2日至T日股价涨跌幅，股票收益
df['正股T-2日至T日涨跌幅'] = df.apply(lambda x : (x['T日均价'] - x['T-2日均价'])/x['T-2日均价'], axis=1)
for d, v in year.items():
    tf = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]
    profit_rt = tf.apply(lambda x : (x['T日均价'] - x['T-2日均价'])/x['T-2日均价'], axis=1).mean()
    profit_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x['T-2日均价']), axis=1).mean()
    print("{}\t 正股T-2日至T日涨跌幅: {:.2f}% \t 正股盈利: {:.2f}".format(d, profit_rt*100, profit_st))


整体	 正股T-2日至T日涨跌幅: -1.21% 	 正股盈利: -107.52
2019	 正股T-2日至T日涨跌幅: 0.37% 	 正股盈利: 76.52
2020	 正股T-2日至T日涨跌幅: -1.25% 	 正股盈利: -103.38
2021	 正股T-2日至T日涨跌幅: -1.42% 	 正股盈利: -118.30
2022	 正股T-2日至T日涨跌幅: -2.26% 	 正股盈利: -257.50


In [7]:
# 转换价值和转股溢价率
df['转股价值'] = df.apply(lambda x : 100/x['初始转股价'] * x['可转债上市日正股收盘价'], axis=1)
for d, v in year.items():
    tf = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]
    price_avg = tf['上市首日收盘价'].mean()
    convertion_price = tf['转股价值'].mean()
    premium_ratio = tf.apply(lambda x : (x['上市首日收盘价']-x['转股价值'])/x['转股价值'], axis=1).mean() * 100
    print("{}\t 可转债上市首日 平均价格: {:.2f}\t 平均转股价值: {:.2f}\t 平均转股溢价率: {:.2f}%".format(d, price_avg, convertion_price, premium_ratio))


整体	 可转债上市首日 平均价格: 119.82	 平均转股价值: 100.36	 平均转股溢价率: 21.19%
2019	 可转债上市首日 平均价格: 109.44	 平均转股价值: 101.18	 平均转股溢价率: 8.93%
2020	 可转债上市首日 平均价格: 119.10	 平均转股价值: 103.37	 平均转股溢价率: 16.56%
2021	 可转债上市首日 平均价格: 119.27	 平均转股价值: 100.20	 平均转股溢价率: 19.97%
2022	 可转债上市首日 平均价格: 130.18	 平均转股价值: 95.01	 平均转股溢价率: 40.04%


In [8]:
# 股价波动
t = {
    'T-3' : 'T-3日',
    'T-2' : 'T-2日',
    'T-1' : 'T-1日', 
    'T' : 'T日'
}
print("涨跌幅\t T-3日\t T-2日\t T-1日\t T日")
for d, v in year.items():
    tf = df[(df['上市公告日期']>=v[0]) & (df['上市公告日期']<=v[1])]
    limit_avg = [ '{:.2f}%'.format(tf[value + '涨跌幅'].mean()) for key, value in t.items() ]
    print("{}\t {}".format(d, ' \t'.join(limit_avg)))

涨跌幅	 T-3日	 T-2日	 T-1日	 T日
整体	 0.53% 	1.85% 	0.24% 	-1.60%
2019	 0.56% 	1.40% 	0.50% 	-0.23%
2020	 0.49% 	1.97% 	0.37% 	-1.72%
2021	 0.50% 	2.13% 	-0.14% 	-1.20%
2022	 0.58% 	1.78% 	0.19% 	-2.95%


In [9]:
# 正股不同市值对抢权策略的影响
market_value = {
    '小于50亿' : [0,50],
    '50-100亿' : [50, 100],
    '100-500亿' : [100, 500],
    '500亿以上' : [500, 100000]
}
df['正股市值'] = df.apply(lambda x : x['正股总股本（万)'] * x['T-3日收盘'] / 10000, axis=1)

print("{:10s}\t {:<10s}\t {:>10s}\t {:10s}\t {:10s}\t {:10s}".format("正股市值","T-2日至T日", "T日","转债盈亏","正股盈亏","总收益"))
for d, v in market_value.items():
    tf = df[(df['正股市值'] < v[1]) & (df['正股市值'] >= v[0])]
    #for idx, row in tf.iterrows():
        #print(row['证券名称'], row['正股总股本（万)'], row['T-3日收盘'], row['正股总股本（万)'] * row['T-3日收盘'] / 10000)
    limit_avg_1 = tf.apply(lambda x : (x['T日均价']-x['T-2日均价'])/x['T-2日均价'], axis=1).mean()
    limit_avg_2 = tf['T日涨跌幅'].mean()
    profit_cb = tf['可转债上市当天成交均价'].apply(lambda x : x-100).mean() * 10
    profit_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x['T-2日均价']), axis=1).mean()
    profit_avg = profit_cb + profit_st
    print("{:10s}\t {:<10.2%}\t {:>10.2f}%\t {:<10.2f}\t {:<10.2f}\t {:<10.2f}".format(d, limit_avg_1, limit_avg_2, profit_cb, profit_st, profit_avg))

正股市值      	 T-2日至T日   	         T日	 转债盈亏      	 正股盈亏      	 总收益       
小于50亿     	 -1.45%    	      -2.14%	 186.59    	 -115.93   	 70.66     
50-100亿   	 -1.30%    	      -1.55%	 181.37    	 -81.67    	 99.70     
100-500亿  	 -1.02%    	      -1.29%	 217.11    	 -114.84   	 102.27    
500亿以上    	 -0.81%    	      -1.09%	 245.51    	 -124.19   	 121.32    


In [10]:
# T-3日至T-5日股价涨跌幅对抢权策略的影响
limit_value = {
    '大于10%' : [0.1, 10],
    '5%至10%' : [0.05, 0.1],
    '0%至5%' : [0, 0.05],
    '-5%至0%' : [-0.05, 0],
    '-10%至-5%' : [-0.1, -0.05],
    '小于-10%' : [-1, -0.1]
}
df['T-3日至T-5日股价涨跌幅'] = df.apply(lambda x : (x['T-3日均价']-x['T-5日均价'])/x['T-5日均价'], axis=1)
print("{:10s}\t {:<10s}\t {:>10s}\t {:<10s}\t {:<10s}\t {:<10s}\t {}".format("T-3至T-5日", "T-2至T日", "T日","转债盈亏","正股盈亏","总盈亏","数量"))
for d, v in limit_value.items():
    tf = df[ (df['T-3日至T-5日股价涨跌幅'] < v[1]) & (df['T-3日至T-5日股价涨跌幅'] >= v[0]) ]
    #tf.apply(lambda x : print(x['证券代码'], x['T-3日均价'], x['T-5日均价'], (x['T-3日均价'] - x['T-5日均价'])/x['T-5日均价']), axis=1)
    limit_avg_1 = tf.apply(lambda x : (x['T日均价']-x['T-2日均价'])/x['T-2日均价'], axis=1).mean()
    limit_avg_2 = tf['T日涨跌幅'].mean()
    count = tf['T日涨跌幅'].count()
    profit_cb = tf['可转债上市当天成交均价'].apply(lambda x : x-100).mean() * 10
    profit_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x['T-2日均价']), axis=1).mean()
    profit_avg = profit_cb + profit_st
    print("{:10s}\t {:<10.2%}\t {:>10.2f}%\t {:<10.2f}\t {:<10.2f}\t {:<10.2f}\t {}".format(d, limit_avg_1, limit_avg_2, profit_cb, profit_st, profit_avg, count))

T-3至T-5日  	 T-2至T日    	         T日	 转债盈亏      	 正股盈亏      	 总盈亏       	 数量
大于10%     	 -1.62%    	      -0.75%	 117.41    	 -127.06   	 -9.65     	 12
5%至10%    	 -1.19%    	      -1.09%	 198.78    	 -174.58   	 24.20     	 46
0%至5%     	 -1.65%    	      -2.01%	 208.03    	 -164.06   	 43.97     	 245
-5%至0%    	 -1.13%    	      -1.33%	 186.19    	 -88.24    	 97.95     	 216
-10%至-5%  	 1.49%     	      -1.19%	 296.22    	 277.10    	 573.32    	 31
小于-10%    	 0.31%     	      -2.30%	 165.60    	 28.06     	 193.66    	 8


In [11]:
# 安全边际对抢权策略的影响
limit_value = {
    '1%以下' : [-10000, 0.01],
    '1-2%' : [0.01, 0.02],
    '2-3%' : [0.02, 0.03],
    '3-4%' : [0.03, 0.04],
    '4-5%' : [0.04, 0.05],
    '大于5%' : [0.05, 10000]
}

c = {
    '预计转债上市当天涨幅20%' : 0.2,
    '预计转债上市当天涨幅30%' : 0.3, 
}

for d, v in c.items():
    print(d)
    print("{:10s}\t {:<10s}\t {:>10s}\t {:<10s}\t {:<10s}\t {:<10s}\t {}".format("安全边际", "T-2至T日", "T日","转债盈亏","正股盈亏","总盈亏","数量"))
    df['安全边际'] = df.apply(lambda x : (10 * 100 * v / (1000/x['每股配售（元）']))/x['T-2日均价'], axis=1)
    for key, value in limit_value.items():
        tf = df[ (df['安全边际'] >= value[0]) & (df['安全边际'] <= value[1])]
        limit_avg_1 = tf.apply(lambda x : (x['T日均价']-x['T-2日均价'])/x['T-2日均价'], axis=1).mean()
        limit_avg_2 = tf['T日涨跌幅'].mean()
        count = tf['T日涨跌幅'].count()
        profit_cb = tf['可转债上市当天成交均价'].apply(lambda x : x-100).mean() * 10
        profit_st = tf.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x['T-2日均价']), axis=1).mean()
        profit_avg = profit_cb + profit_st
        print("{:10s}\t {:<10.2%}\t {:>10.2f}%\t {:<10.2f}\t {:<10.2f}\t {:<10.2f}\t {}".format(key, limit_avg_1, limit_avg_2, profit_cb, profit_st, profit_avg, count))

预计转债上市当天涨幅20%
安全边际      	 T-2至T日    	         T日	 转债盈亏      	 正股盈亏      	 总盈亏       	 数量
1%以下      	 -0.12%    	      -0.41%	 291.68    	 -24.21    	 267.47    	 110
1-2%      	 -1.24%    	      -1.72%	 219.07    	 -159.67   	 59.40     	 191
2-3%      	 -1.36%    	      -1.60%	 163.98    	 -116.18   	 47.80     	 139
3-4%      	 -0.89%    	      -1.62%	 144.96    	 -56.07    	 88.89     	 64
4-5%      	 -3.81%    	      -3.54%	 130.68    	 -177.47   	 -46.79    	 23
大于5%      	 -2.95%    	      -3.72%	 104.50    	 -97.27    	 7.23      	 31
预计转债上市当天涨幅30%
安全边际      	 T-2至T日    	         T日	 转债盈亏      	 正股盈亏      	 总盈亏       	 数量
1%以下      	 0.13%     	      -0.51%	 287.99    	 40.96     	 328.95    	 55
1-2%      	 -0.44%    	      -0.95%	 282.83    	 -94.27    	 188.56    	 115
2-3%      	 -1.57%    	      -1.79%	 195.14    	 -187.58   	 7.56      	 131
3-4%      	 -1.37%    	      -1.50%	 167.96    	 -123.15   	 44.81     	 108
4-5%      	 -2.08%    	      -2.66%	 157.90    	 -134.34

In [12]:
df['转债盈亏'] = df.apply(lambda x : (x['可转债上市当天成交均价']-100) * 10, axis=1)
df['正股盈亏'] = df.apply(lambda x : 1000 / x['每股配售（元）'] * (x['T日均价'] - x['T-2日均价']), axis=1)
df['总盈亏'] = df.apply(lambda x : x['转债盈亏'] + x['正股盈亏'], axis=1)

tf = pandas.DataFrame()
tf['证券代码'] = df['证券代码']
tf['证券名称'] = df['证券名称']
tf['转债盈亏'] = df['转债盈亏']
tf['正股盈亏'] = df['正股盈亏']
tf['总盈亏'] = df['总盈亏']

tf['股东配售比例'] = df['股东配售比例']
tf['正股市值'] = df['正股市值']
tf['T-3日至T-5日股价涨跌幅'] = df['T-3日至T-5日股价涨跌幅']
tf['安全边际'] = df['安全边际']
tf['正股T-2日至T日涨跌幅'] = df['正股T-2日至T日涨跌幅']
tf['可转债上市当天涨跌幅'] = df.apply(lambda x : (x['可转债上市当天成交均价']-100)/100, axis=1)
print(tf.corr())


tf.to_excel(current_path + '/../data/statistics.xlsx', index=False)



                    转债盈亏      正股盈亏       总盈亏    股东配售比例      正股市值  \
转债盈亏            1.000000 -0.010355  0.227495  0.275394  0.070642   
正股盈亏           -0.010355  1.000000  0.971371  0.030546  0.069028   
总盈亏             0.227495  0.971371  1.000000  0.095174  0.084004   
股东配售比例          0.275394  0.030546  0.095174  1.000000  0.088963   
正股市值            0.070642  0.069028  0.084004  0.088963  1.000000   
T-3日至T-5日股价涨跌幅 -0.052707 -0.105238 -0.115006  0.063986  0.000502   
安全边际           -0.292556 -0.007969 -0.077265 -0.156074 -0.173309   
正股T-2日至T日涨跌幅    0.073041  0.780167  0.777104 -0.026956  0.066081   
可转债上市当天涨跌幅      1.000000 -0.010355  0.227495  0.275394  0.070642   

                T-3日至T-5日股价涨跌幅      安全边际  正股T-2日至T日涨跌幅  可转债上市当天涨跌幅  
转债盈亏                 -0.052707 -0.292556      0.073041    1.000000  
正股盈亏                 -0.105238 -0.007969      0.780167   -0.010355  
总盈亏                  -0.115006 -0.077265      0.777104    0.227495  
股东配售比例                0.063986 -0.156074   